# TODO:
* Adapt PlotCurves() to work with tensorflow functions/methods
* RQ2-Bert yap

Following: https://github.com/strongio/keras-bert/blob/master/keras-bert.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

In [4]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [5]:
from src.keras_bert import convert_text_to_examples, \
                           create_tokenizer_from_hub_module, \
                           convert_examples_to_features, \
                           initialize_vars, \
                           build_model, \
                           BertLayer

W0901 23:56:22.360902 140486611460224 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/Master_Thesis/src/keras_bert.py:12: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score as scikit_f1_score

In [7]:
# Initialize session
sess = tf.Session()

#### Custom Functions

In [8]:
from src.callbacks import PlotCurves
from src.custom_functions import f1_micro, f1_macro
from src.load_data import load_data

### Load Data

In [9]:
train_data, valid_data, test_data, metadata = load_data()

In [10]:
max([len(sentence['sentence']) 
     for article in (train_data + valid_data + test_data)
     for sentence in article['sentences']])

639

### Prepare data

In [1]:
max_seq_length = 512
if max_seq_length > 512:
    print('!!!!!!! WARNING: BERT does not accept lenght > 512')
    max_seq_length = 512

In [14]:
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

In [15]:
def split_data(data_, max_seq_length, bert_path, to_categorize):
    
    tokenizer = create_tokenizer_from_hub_module(bert_path)
    
    # !!! For BERT input, each sentence should be in an array
    X = np.array([[" ".join(sentence['sentence'].replace('\n', '').strip().split()[0:max_seq_length])]
                  for article in data_ 
                  for sentence in article['sentences']], dtype=object)

    y = [sentence['label'] 
                  for article in data_
                  for sentence in article['sentences']]
    
    examples_ = convert_text_to_examples(X, y)
    
    (input_ids, input_masks, segment_ids, labels_) = \
            convert_examples_to_features(tokenizer, examples_, max_seq_length=max_seq_length)
    
    if to_categorize:
        labels_ = to_categorical(labels_)
    
    return [input_ids, input_masks, segment_ids], labels_

In [16]:
X_tra, y_tra = split_data(train_data, max_seq_length, bert_path, True)
X_val, y_val = split_data(valid_data, max_seq_length, bert_path, True)
X_test, y_test = split_data(test_data, max_seq_length, bert_path, False)

W0901 23:56:44.645421 140486611460224 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/Master_Thesis/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Converting examples to features: 100%|██████████| 441/441 [00:00<00:00, 2548.65it/s]


## Build Model

In [17]:
def build_flat_model_0(max_seq_length):
    
    in_id = Input(shape=(max_seq_length,), name="input_ids")
    in_mask = Input(shape=(max_seq_length,), name="input_masks")
    in_segment = Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = BertLayer(n_fine_tune_layers=3,
                            pooling='mean'
                            bert_path=bert_path)(bert_inputs)
    
    dense = Dense(512, activation="relu")(bert_output)
    dense = Dense(256, activation="relu")(dense)
    
    pred = Dense(2, activation="sigmoid")(dense)

    return Model(inputs=bert_inputs, outputs=pred)

In [18]:
learningrate=0.001
optimizer = Adam(lr=learningrate)
optimizer_str = 'Adam'
loss = 'binary_crossentropy'
metrics = ['acc', f1_macro, f1_micro]

In [19]:
model_name = 'RQ1_bert_flat_model_0_' + optimizer_str + '_lr_' + str(learningrate) + '_loss_' + loss # + '_with_learnin_rate_reduction'
model = build_flat_model_0(max_seq_length)
model.summary()

model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

W0901 23:57:11.568778 140486611460224 deprecation.py:506] From /home/aorus/workspaces/simge/Master_Thesis/.env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]            

### Train

In [22]:
# Instantiate variables
initialize_vars(sess)

In [23]:
## Flat Model Fit
model.fit(X_tra, y_tra, epochs=20, batch_size=32, validation_data=(X_val, y_val))
#           callbacks=[PlotCurves(model_name=model_name)])

Train on 3582 samples, validate on 399 samples
3582/3582 [==============================] - 126s 35ms/sample - loss: 0.4220 - acc: 0.7997 - f1_macro: 0.7197 - f1_micro: 0.7925 - val_loss: 0.3110 - val_acc: 0.8596 - val_f1_macro: 0.8503 - val_f1_micro: 0.8654


### Validation Set


In [24]:
import os
model_name

'RQ1_bert_flat_model_0_Adam_lr_0.001_loss_binary_crossentropy'

In [25]:
results_file = os.path.join('Model/' + model_name, 'model_results_file.txt')
        
with open(results_file, 'w') as f:
    f.write('\n----------------\n')

In [27]:
y_val_true = np.array([sentence['label'] for article in valid_data 
                                 for sentence in article['sentences']])

y_val_preds = model.predict(X_val, batch_size=32)
y_val_preds = np.argmax(y_val_preds, axis=1)

print(classification_report(y_val_true, y_val_preds))
with open(results_file, 'a') as f:
    f.write(classification_report(y_val_true, y_val_preds))
    f.write('\n')

              precision    recall  f1-score   support

           0       0.93      0.85      0.89       269
           1       0.74      0.88      0.80       130

   micro avg       0.86      0.86      0.86       399
   macro avg       0.84      0.86      0.85       399
weighted avg       0.87      0.86      0.86       399



### Test

In [29]:
y_preds = model.predict(X_test, batch_size=32)
y_preds = np.argmax(y_preds, axis=1)

In [30]:
### build_flat_model_1 results
print(classification_report(y_test, y_preds))
with open(results_file, 'a') as f:
    f.write(classification_report(y_test, y_preds))
    f.write('\n')

              precision    recall  f1-score   support

           0       0.91      0.87      0.89       325
           1       0.68      0.77      0.72       116

   micro avg       0.84      0.84      0.84       441
   macro avg       0.80      0.82      0.81       441
weighted avg       0.85      0.84      0.85       441



In [31]:
scikit_f1_score(y_test, y_preds,average='macro')

0.8059931779782589

# Oylesine birden alinan sonuclar

##### Validation

In [41]:
y_val_true = np.array([sentence['label'] for article in valid_data 
                                 for sentence in article['sentences']])

y_val_preds = model.predict(X_val, batch_size=32)
y_val_preds = np.argmax(y_val_preds, axis=1)

print(classification_report(y_val_true, y_val_preds))
with open(results_file, 'a') as f:
    f.write(classification_report(y_val_true, y_val_preds))
    f.write('\n')

              precision    recall  f1-score   support

           0       0.93      0.85      0.89       269
           1       0.73      0.87      0.80       130

   micro avg       0.85      0.85      0.85       399
   macro avg       0.83      0.86      0.84       399
weighted avg       0.87      0.85      0.86       399



In [44]:
scikit_f1_score(y_val_true, y_val_preds,average='macro')

0.841467090480627

##### Test

In [48]:
y_preds = model.predict(X_test, batch_size=32)
y_preds = np.argmax(y_preds, axis=1)

In [49]:
### build_flat_model_1 results
print(classification_report(y_test, y_preds))
with open(results_file, 'a') as f:
    f.write(classification_report(y_test, y_preds))
    f.write('\n')

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       325
           1       0.75      0.78      0.76       116

   micro avg       0.87      0.87      0.87       441
   macro avg       0.83      0.84      0.84       441
weighted avg       0.88      0.87      0.87       441



In [50]:
scikit_f1_score(y_test, y_preds, average='macro')

0.8388746803069054